# 06 - Classification Random Forest

Entraînement d'un modèle MLlib pour classifier les phases de vol.

## Configuration

In [1]:
from pyspark.sql.functions import col, sum as _sum, when
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from config import get_s3_path, create_spark_session

SILVER_ML_PATH = get_s3_path("silver", "flights_ml")
GOLD_MODEL_PATH = get_s3_path("gold", "models", "rf_flight_phase")
GOLD_PREDICTIONS_PATH = get_s3_path("gold", "predictions", "flight_phase")

spark = create_spark_session("RandomForestClassification")

print(f"✅ Input: {SILVER_ML_PATH}")

✅ Configuration chargée depuis .env
:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.spark#spark-hadoop-cloud_2.12 added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e123ade5-7281-4b45-ae15-dd258e157326;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-hadoop-cloud_2.12;3.5.3 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.c

✅ Spark Session 'RandomForestClassification' configurée
✅ Input: s3a://datalake/silver/flights_ml


## Lecture des données

In [2]:
feature_columns = [
    "altitude_meters", "velocity_kmh", "altitude_change", "velocity_change",
    "rolling_avg_altitude", "rolling_std_altitude", "rolling_avg_velocity"
]

df = spark.read.format("delta").load(SILVER_ML_PATH)
df_clean = df.filter(col("flight_phase").isNotNull()).fillna(0, subset=feature_columns)

print(f"📊 {df_clean.count():,} lignes")
df_clean.groupBy("flight_phase").count().orderBy("count", ascending=False).show()

26/01/23 15:51:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
26/01/23 15:52:39 ERROR NonFateSharingFuture: Failed to get result from future  
scala.runtime.NonLocalReturnControl
26/01/23 15:52:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


📊 813,930 lignes


+------------+------+
|flight_phase| count|
+------------+------+
|  TRANSITION|454844|
|      CRUISE|204211|
|     DESCENT| 82364|
|       CLIMB| 54754|
|     TAKEOFF| 15856|
|      GROUND|  1901|
+------------+------+



## Pipeline MLlib

In [3]:
label_indexer = StringIndexer(inputCol="flight_phase", outputCol="label", handleInvalid="skip")
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_raw", handleInvalid="skip")
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withStd=True, withMean=False)
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=10, seed=42)
label_converter = IndexToString(inputCol="prediction", outputCol="predicted_label", labels=label_indexer.fit(df_clean).labels)

pipeline = Pipeline(stages=[label_indexer, vector_assembler, scaler, rf, label_converter])

print("✅ Pipeline créé")

✅ Pipeline créé


## Entraînement

In [ ]:
train_df, test_df = df_clean.randomSplit([0.8, 0.2], seed=42)

print(f"📊 Train: {train_df.count():,} | Test: {test_df.count():,}")
print("🚀 Entraînement...")

model = pipeline.fit(train_df)

print("✅ Modèle entraîné")

## Évaluation

In [ ]:
predictions = model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"🎯 Accuracy: {accuracy:.4f}")
print(f"🎯 F1 Score: {f1:.4f}")

26/01/23 14:03:15 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
26/01/23 14:03:17 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


🎯 Accuracy: 0.9422
🎯 F1 Score: 0.9414


## Matrice de confusion

In [ ]:
predictions.groupBy("flight_phase", "predicted_label").count() \
    .orderBy("flight_phase", "predicted_label").show(50)

26/01/23 14:03:18 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
26/01/23 14:03:20 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB


+------------+---------------+-----+
|flight_phase|predicted_label|count|
+------------+---------------+-----+
|       CLIMB|          CLIMB| 1731|
|       CLIMB|        TAKEOFF|   69|
|      CRUISE|         CRUISE| 7237|
|      CRUISE|        DESCENT|   36|
|      CRUISE|     TRANSITION|   39|
|     DESCENT|         CRUISE|    5|
|     DESCENT|        DESCENT| 2113|
|     DESCENT|     TRANSITION|    7|
|      GROUND|         GROUND|   22|
|      GROUND|     TRANSITION|   31|
|     TAKEOFF|          CLIMB|    4|
|     TAKEOFF|        TAKEOFF|  382|
|  TRANSITION|         CRUISE|  750|
|  TRANSITION|        DESCENT|   65|
|  TRANSITION|         GROUND|    8|
|  TRANSITION|     TRANSITION| 5048|
+------------+---------------+-----+



## Importance des features

In [ ]:
import pandas as pd

rf_model = model.stages[3]
importances = rf_model.featureImportances.toArray()

pd.DataFrame({
    "feature": feature_columns,
    "importance": importances
}).sort_values("importance", ascending=False)

,feature,importance
2,altitude_change,0.383648
0,altitude_meters,0.194921
4,rolling_avg_altitude,0.158527
5,rolling_std_altitude,0.138574
1,velocity_kmh,0.065699
3,velocity_change,0.029685
6,rolling_avg_velocity,0.028947


## Sauvegarde

In [ ]:
model.write().overwrite().save(GOLD_MODEL_PATH)
print(f"✅ Modèle: {GOLD_MODEL_PATH}")

predictions.select(
    "event_timestamp", "icao24", "callsign", "flight_phase", "predicted_label", "probability"
).write.format("delta").mode("overwrite").save(GOLD_PREDICTIONS_PATH)

print(f"✅ Prédictions: {GOLD_PREDICTIONS_PATH}")

✅ Modèle: s3a://datalake/gold/models/rf_flight_phase


26/01/23 14:03:25 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


✅ Prédictions: s3a://datalake/gold/predictions/flight_phase
